# RFE

In [1]:
import pandas as pd
import numpy as np
import imblearn
from imblearn.pipeline import make_pipeline as make_pipeline_imbfinal 
from imblearn.over_sampling import SMOTE
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB 
#from sklearn import svm
from sklearn.metrics import *
import pickle
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv('ckd.csv')

Buildinig models with features retrived in Boruta.

In [3]:
df_train,df_test = train_test_split(df,train_size=0.7,random_state=42)
drop_col_list=['RBC_normal','PCC_present','BA_present','CAD_yes','Ane_yes' , 'PC_normal' , 'Appet_good' ,
               'age' , 'pot' , 'wc' , 'PE_yes' , 'bp' , 'su' ]
x_train=df_train.iloc[:,:24]
x_train.drop(drop_col_list,axis=1,inplace=True)
y_train=df_train['Classification_ckd']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=df_test.iloc[:,:24]
x_test.drop(drop_col_list,axis=1,inplace=True)
y_test=df_test['Classification_ckd']
scaler.fit(x_test)
x_test_sc=scaler.transform(x_test)

Executing **RFE**

In [4]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
X = x_train_sc
y = y_train
estimator = SVC(kernel="linear")
selector = RFE(estimator, 5, step=1)
selector = selector.fit(X, y)
selector.support_ 

array([ True,  True, False, False,  True, False,  True, False, False,
       False,  True], dtype=bool)

In [5]:
selector.ranking_

array([1, 1, 4, 7, 1, 3, 1, 2, 5, 6, 1])

In [6]:
x_train.columns[selector.support_]

Index(['sg', 'al', 'sc', 'hemo', 'DM_yes'], dtype='object')

In [7]:
df_train,df_test = train_test_split(df,train_size=0.7,random_state=42)
x_train=df_train.iloc[:,:24]
y_train=df_train['Classification_ckd']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=df_test.iloc[:,:24]
y_test=df_test['Classification_ckd']
scaler.fit(x_test)
x_test_sc=scaler.transform(x_test)

In [10]:
X = x_train_sc
y = y_train
estimator = SVC(kernel="linear")
selector = RFE(estimator, 10, step=1)
selector = selector.fit(X, y)
selector.support_ 

array([False, False,  True,  True, False,  True,  True,  True, False,
       False,  True,  True, False, False, False, False, False, False,
       False,  True, False,  True,  True, False], dtype=bool)

Retriving important features after executing RFE.

In [11]:
x_train.columns[selector.support_]

Index(['sg', 'al', 'bgr', 'bu', 'sc', 'hemo', 'pcv', 'DM_yes', 'Appet_good',
       'PE_yes'],
      dtype='object')

Building model with newly retrived features.

In [13]:
column_list = x_train.columns[selector.support_]
x_train=df_train.iloc[:,1:24]
x_train= x_train[column_list]
y_train=df_train['Classification_ckd']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=df_test.iloc[:,1:24]
x_test = x_test[column_list]
y_test=df_test['Classification_ckd']
x_test_sc=scaler.transform(x_test)

Running SVC.

In [15]:
svc = SVC(C=0.025, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)
svc.fit(x_train_sc, y_train)

SVC(C=0.025, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

In [16]:
accuracy =[]
model_name =[]
dataset=[]
f1score = []
precision = []
recall = []
true_positive =[]
false_positive =[]
true_negative =[]
false_negative =[]

In [17]:
prediction = svc.predict(x_train_sc)
f1 = f1_score(y_train, prediction)
p = precision_score(y_train, prediction)
r = recall_score(y_train, prediction)
a = accuracy_score(y_train, prediction)
cm = confusion_matrix(y_train, prediction)
tp = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tn = cm[1][1]
model_name.append('Support Vector Classifier')
dataset.append('Training')
f1score.append(f1)
precision.append(p)
recall.append(r)
accuracy.append(a)
true_positive.append(tp) 
false_positive.append(fp)
true_negative.append(tn) 
false_negative.append(fn)
cm

array([[106,   0],
       [  3, 171]], dtype=int64)

In [18]:
prediction = svc.predict(x_test_sc)
f1 = f1_score(y_test,  prediction)
p = precision_score(y_test,  prediction)
r = recall_score(y_test,  prediction)
a = accuracy_score(y_test,  prediction)
cm = confusion_matrix(y_test,  prediction)
tp = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tn = cm[1][1]
model_name.append('Support Vector Classifier')
dataset.append('Testing')
f1score.append(f1)
precision.append(p)
recall.append(r)
accuracy.append(a)
true_positive.append(tp) 
false_positive.append(fp)
true_negative.append(tn) 
false_negative.append(fn)
cm

array([[44,  0],
       [ 1, 75]], dtype=int64)